<a href="https://colab.research.google.com/github/Sandli07goyal/Cantilever/blob/main/Music_Generation_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from music21 import converter, instrument, note, chord, stream

In [2]:
def parse_midi(file_path):
    midi = converter.parse(file_path)
    notes = []
    parts = instrument.partitionByInstrument(midi)
    for part in parts.parts:
        if 'Piano' in str(part):
            notes_to_parse = part.recurse()
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

def prepare_sequences(notes, sequence_length):
    pitch_names = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitch_names))

    input_seq = []
    output_seq = []

    for i in range(0, len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        input_seq.append([note_to_int[char] for char in sequence_in])
        output_seq.append(note_to_int[sequence_out])

    n_vocab = len(set(notes))
    input_seq = np.reshape(input_seq, (len(input_seq), sequence_length, 1))
    input_seq = input_seq / float(n_vocab)
    output_seq = tf.keras.utils.to_categorical(output_seq)

    return input_seq, output_seq, n_vocab, note_to_int

In [3]:
def create_model(input_shape, n_vocab):
    model = models.Sequential([
        layers.LSTM(256, input_shape=(input_shape[1], input_shape[2]), return_sequences=True),
        layers.Dropout(0.3),
        layers.LSTM(256, return_sequences=False),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(n_vocab, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [4]:
def train_model(model, input_seq, output_seq, epochs=100):
    model.fit(input_seq, output_seq, epochs=epochs, batch_size=64)
    model.save('music_generator_model.h5')

In [5]:
def generate_notes(model, input_seq, note_to_int, n_vocab, num_notes):
    int_to_note = dict((number, note) for note, number in note_to_int.items())
    start = np.random.randint(0, len(input_seq) - 1)
    pattern = input_seq[start]
    prediction_output = []

    for note_index in range(num_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = np.append(pattern, index)
        pattern = pattern[1:]

    return prediction_output

In [6]:
def create_midi(prediction_output, file_name="output.mid"):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            chord_notes = pattern.split('.')
            notes = []
            for current_note in chord_notes:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=file_name)

In [7]:
notes = parse_midi('path_to_your_midi_file.mid')
sequence_length = 100
input_seq, output_seq, n_vocab, note_to_int = prepare_sequences(notes, sequence_length)

model = create_model(input_seq.shape, n_vocab)
train_model(model, input_seq, output_seq, epochs=200)

prediction_output = generate_notes(model, input_seq, note_to_int, n_vocab, num_notes=500)
create_midi(prediction_output, file_name="generated_music.mid")


FileNotFoundError: Cannot find file in path_to_your_midi_file.mid